In [ ]:





48
A. Prediction at Unobserved Locations
A Prediction at Unobserved Locations
The following is an example of predictions at unobserved locations and times.
For brevity, most of the function outputs have been omitted in the Appendices.
A.1 Load Data
Let us first load relevant libraries and data.
> ##libraries
> library(SpatioTemporal)
> library(plotrix)
> ##load data
> data(mesa.data.raw)
> data(mesa.model)
> data(est.mesa.model)
A.2 Setup and Study the Data
We then setup data structures — dropping the spatio-temporal covariate and
all observations at two sites — and study the resulting data.
> mesa.data.raw$obs <-
mesa.data.raw$obs[,!(colnames(mesa.data.raw$obs) %in%
c("60595001", "LC003"))]
> mesa.data <- with(mesa.data.raw,
createSTdata(obs, X, n.basis=2))
Let us also expand the temporal trends to every week instead of every 2-
weeks, giving us unobserved time-points at which to predict.
> mesa.data.org <- mesa.data
> T <- with(mesa.data$trend, seq(min(date), max(date), by=7))
> mesa.data <- updateTrend(mesa.data, n.basis=2, extra.dates=T)
Studying the reduced data structure, we see that 2 locations lack observations,
there are additional time-points, and no spatio-temporal covariates.
49
Tutorial for SpatioTemporal
> print(mesa.data)
STdata-object with:
No. locations: 25 (observed: 23)
No. time points: 559 (observed: 280)
No. obs: 4229
Trend with 2 basis function(s):
[1] "V1" "V2"
with dates:
1999-01-13 to 2009-09-23
12 covariate(s):
[1] "ID" "x"
[3] "y" "long"
[5] "lat" "type"
[7] "log10.m.to.a1" "log10.m.to.a2"
[9] "log10.m.to.a3" "log10.m.to.road"
[11] "km.to.coast" "s2000.pop.div.10000"
No spatio-temporal covariates.
All sites:
AQS FIXED
20 5
Observed:
AQS FIXED
19 4
For AQS:
Number of obs: 3910
Dates: 1999-01-13 to 2009-09-23
For FIXED:
Number of obs: 319
Dates: 2005-12-07 to 2009-07-01
> print(mesa.data.org)
STdata-object with:
No. locations: 25 (observed: 23)
No. time points: 280 (observed: 280)
No. obs: 4229
50
A.2 Setup and Study the Data
Trend with 2 basis function(s):
[1] "V1" "V2"
with dates:
1999-01-13 to 2009-09-23
12 covariate(s):
[1] "ID" "x"
[3] "y" "long"
[5] "lat" "type"
[7] "log10.m.to.a1" "log10.m.to.a2"
[9] "log10.m.to.a3" "log10.m.to.road"
[11] "km.to.coast" "s2000.pop.div.10000"
No spatio-temporal covariates.
All sites:
AQS FIXED
20 5
Observed:
AQS FIXED
19 4
For AQS:
Number of obs: 3910
Dates: 1999-01-13 to 2009-09-23
For FIXED:
Number of obs: 319
Dates: 2005-12-07 to 2009-07-01
We can also compare the smooth temporal trends in the two models
> par(mfrow=c(2,1), mar=c(2,2,2,1))
> plot(mesa.data$trend$date, mesa.data$trend$V1,
xlab="", ylab="", main="Trend 1")
> points(mesa.data.org$trend$date, mesa.data.org$trend$V1,
col="red", pch=3)
> plot(mesa.data$trend$date, mesa.data$trend$V2,
xlab="", ylab="", main="Trend 2")
> points(mesa.data.org$trend$date, mesa.data.org$trend$V2,
col="red", pch=3)
51
Tutorial for SpatioTemporal
The trends are identical, since they are based on observations from the same
23 locations.
We then create a STmodel for the reduced data-set (Using the model specification
in mesa.model).
> mesa.model.1 <- createSTmodel(mesa.data,
LUR=mesa.model$LUR.list, cov.beta=mesa.model$cov.beta,
cov.nu=mesa.model$cov.nu,
locations=mesa.model$locations.list)
> mesa.model.2 <- createSTmodel(mesa.data.org,
LUR=mesa.model$LUR.list, cov.beta=mesa.model$cov.beta,
cov.nu=mesa.model$cov.nu,
locations=mesa.model$locations.list, strip=TRUE)
Studying the models,
> print(mesa.model.1)
> print(mesa.model.2)
we note that they contain different number of locations
> str(loglikeSTdim(mesa.model.1))
List of 12
$ T : int 280
$ m : int 3
$ n : int 25
$ n.obs : int 23
$ p : Named int [1:3] 4 2 2
..- attr(*, "names")= chr [1:3] "const" "V1" "V2"
$ L : int 0
$ npars.beta.covf: Named int [1:3] 2 2 2
..- attr(*, "names")= chr [1:3] "exp" "exp" "exp"
$ npars.beta.tot : Named int [1:3] 2 2 2
..- attr(*, "names")= chr [1:3] "exp" "exp" "exp"
$ npars.nu.covf : int 2
$ npars.nu.tot : int 4
$ nparam.cov : int 10
$ nparam : int 18
> str(loglikeSTdim(mesa.model.2))
52
A.3 Predictions
List of 12
$ T : int 280
$ m : int 3
$ n : int 23
$ n.obs : int 23
$ p : Named int [1:3] 4 2 2
..- attr(*, "names")= chr [1:3] "const" "V1" "V2"
$ L : int 0
$ npars.beta.covf: Named int [1:3] 2 2 2
..- attr(*, "names")= chr [1:3] "exp" "exp" "exp"
$ npars.beta.tot : Named int [1:3] 2 2 2
..- attr(*, "names")= chr [1:3] "exp" "exp" "exp"
$ npars.nu.covf : int 2
$ npars.nu.tot : int 4
$ nparam.cov : int 10
$ nparam : int 18
and different number of unobserved time-points
> dim(mesa.model.1$trend)
[1] 559 3
> dim(mesa.model.2$trend)
[1] 280 3

A.3 Predictions
Having created data structures that contain some unobserved locations and
time-points we are now ready to compute predictions at all unobserved times
and locations.
In a real world application we would first use estimate.STmodel to estimate
parameters. Here we will just use the covariance-parameters previously
estimated in subsection 4.1,
> x <- coef(est.mesa.model, pars="cov")$par
and compute predictions for the two models (this may take several seconds,
we have omitted variance computations to save time).
53
Tutorial for SpatioTemporal
> E.1 <- predict(mesa.model.1, x, pred.var=FALSE)
> E.2 <- predict(mesa.model.2, x, pred.var=FALSE)
Studying the resulting structures, we see that the first case has computed
predictions at all locations, while the second case only computed predictions
at the 23 locations, and 280 time-points
> colnames(E.1$EX)
[1] "60370002" "60370016" "60370030" "60370031" "60370113"
[6] "60371002" "60371103" "60371201" "60371301" "60371601"
[11] "60371602" "60371701" "60372005" "60374002" "60375001"
[16] "60375005" "60590001" "60590007" "60591003" "L001"
[21] "L002" "LC001" "LC002" "60595001" "LC003"
> str(E.1$EX)
num [1:559, 1:25] 4.55 4.31 4 4.17 4.03 ...
- attr(*, "dimnames")=List of 2
..$ : chr [1:559] "1999-01-13" "1999-01-20" "1999-01-27" "1999-02-03" ...
..$ : chr [1:25] "60370002" "60370016" "60370030" "60370031" ...
> colnames(E.2$EX)
[1] "60370002" "60370016" "60370030" "60370031" "60370113"
[6] "60371002" "60371103" "60371201" "60371301" "60371601"
[11] "60371602" "60371701" "60372005" "60374002" "60375001"
[16] "60375005" "60590001" "60590007" "60591003" "L001"
[21] "L002" "LC001" "LC002"
> str(E.2$EX)
num [1:280, 1:23] 4.55 4 4.03 4.18 3.72 ...
- attr(*, "dimnames")=List of 2
..$ : chr [1:280] "1999-01-13" "1999-01-27" "1999-02-10" "1999-02-24" ...
..$ : chr [1:23] "60370002" "60370016" "60370030" "60370031" ...
The predictions are equal (to within numerical precision).
> range(E.1$EX[rownames(E.2$EX),colnames(E.2$EX)] - E.2$EX)
[1] 0 0
We could also use the original data structure to define additional prediction
locations.

54
A.3 Predictions
> E.3 <- predict(mesa.model.2, x, STdata=mesa.data, pred.var=FALSE)
Here, the trend from mesa.model.2 is used for prediction, with values at
additional time-points computed by mesa.model.2$trend.fnc. Having obtained
predictions at all locations and times in mesa.data, we compare these
to those in E.1.
> colnames(E.3$EX)
[1] "60370002" "60370016" "60370030" "60370031" "60370113"
[6] "60371002" "60371103" "60371201" "60371301" "60371601"
[11] "60371602" "60371701" "60372005" "60374002" "60375001"
[16] "60375005" "60590001" "60590007" "60591003" "L001"
[21] "L002" "LC001" "LC002" "60595001" "LC003"
> str(E.3$EX)
num [1:559, 1:25] 4.55 4.31 4 4.17 4.03 ...
- attr(*, "dimnames")=List of 2
..$ : chr [1:559] "1999-01-13" "1999-01-20" "1999-01-27" "1999-02-03" ...
..$ : chr [1:25] "60370002" "60370016" "60370030" "60370031" ...
> all.equal(E.3,E.1)
[1] TRUE

A.3.1 Temporal Averages
The predict function provides an option for computing temporal averages,
and variances of the temporal averages. The averages can either be over
all time points or over only a few time-points. Here we will ilustrate by
computing temporal averages for each year. The first step is to create a list
where each component gives the dates over which to average, e.g. by splitting
the observations dates between each year.
> LTA <- with(mesa.model.1$trend, split(date,as.POSIXlt(date)$year+1900))
> str(LTA)
List of 11
$ 1999: Date[1:51], format: "1999-01-13" ...
$ 2000: Date[1:52], format: "2000-01-05" ...
$ 2001: Date[1:52], format: "2001-01-03" ...
55
Tutorial for SpatioTemporal
$ 2002: Date[1:52], format: "2002-01-02" ...
$ 2003: Date[1:53], format: "2003-01-01" ...
$ 2004: Date[1:52], format: "2004-01-07" ...
$ 2005: Date[1:52], format: "2005-01-05" ...
$ 2006: Date[1:52], format: "2006-01-04" ...
$ 2007: Date[1:52], format: "2007-01-03" ...
$ 2008: Date[1:53], format: "2008-01-02" ...
$ 2009: Date[1:38], format: "2009-01-07" ...
> lapply(LTA[1:3], range)
$`1999`
[1] "1999-01-13" "1999-12-29"
$`2000`
[1] "2000-01-05" "2000-12-27"
$`2001`
[1] "2001-01-03" "2001-12-26"
To allow for averaging over different time at each location, we need a list
with each element named by the location and containing the temporal list
constructed above.
> ID <- mesa.model.1$locations$ID
> LTA <- rep(list(LTA), length(ID))
> names(LTA) <- ID
The LTA object can now be used as an argument to predict, specifying which
times, at each location, that we should average over.
> E.1.LTA <- predict(mesa.model.1, x, pred.var=FALSE, LTA=LTA)
The resulting predictions now contain a LTA field that provides the averages
and (not shown here) variances.
> head(E.1.LTA$LTA)
EX.mu EX.mu.beta EX
60370002.1 4.156456 4.080504 4.072988
60370002.2 4.060114 3.952042 3.956384
60370002.3 3.972375 3.922069 3.904141
56
A.3 Predictions
60370002.4 3.919108 3.881755 3.876229
60370002.5 3.882735 3.824435 3.827195
60370002.6 3.729919 3.578361 3.562588
The element E.1.LTAoptsLTA.list contains a copy of the LTA option provided
in the call to predict that can be used to check over which time points each
average has been computed.
In this simple case the averages are the same as those obtained by just averaging
the predictions; the main need for seperate computation of averages are
to: 1) get correct variances, accounting for temporal dependencies due to the
temporal basis functions; and 2) account for non-linearities when averaging
over log-Gaussian fields.
> E.1.LTA.alt <- sapply(split(E.1$EX[,1], as.POSIXlt(rownames(E.1$EX))$year),mean)
> cbind(E.1.LTA.alt, E.1.LTA$LTA$EX[1:11])
E.1.LTA.alt
99 4.072988 4.072988
100 3.956384 3.956384
101 3.904141 3.904141
102 3.876229 3.876229
103 3.827195 3.827195
104 3.562588 3.562588
105 3.628571 3.628571
106 3.624019 3.624019
107 3.540996 3.540996
108 3.400788 3.400788
109 3.159180 3.159180
57
Tutorial for SpatioTemporal
